In [1]:
from pyspark.sql.functions import from_json, col
import json
import pandas as pd
from pyspark.sql.functions import explode, split
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, TimestampType, ArrayType, FloatType
import warnings
from pyspark.sql.functions import size
from pyspark.sql.functions import regexp_replace
import numpy as np

# Pull in live game update data

In [2]:
df = spark.read.parquet('/tmp/games')
df.registerTempTable('games')
query = """
create external table my_games
  stored as parquet
  location '/tmp/games_out'
  as
  select * from games
"""
spark.sql(query)

DataFrame[]

In [3]:
games2 = spark.read.parquet('/tmp/games_out')

In [5]:
final_schema = StructType([
        StructField('GameKey', StringType(), True),
        StructField('Week', StringType(), True),
        StructField('AwayTeam', StringType(), True),
        StructField('AwayScore', IntegerType(), True),
        StructField('HomeTeam', StringType(), True),
        StructField('HomeScore', IntegerType(), True),
        StructField('PointSpread', FloatType(), True),
        StructField('OverUnder', FloatType(), True),
        StructField('AwayTeamMoneyLine', IntegerType(), True),
        StructField('HomeTeamMoneyLine', IntegerType(), True)])

In [6]:
extracted_games2 = games2.rdd.map(lambda x: json.loads(x.value)).toDF(schema=final_schema)

In [7]:
pd_games_stream = extracted_games2.toPandas()
pd_games_stream

,GameKey,Week,AwayTeam,AwayScore,HomeTeam,HomeScore,PointSpread,OverUnder,AwayTeamMoneyLine,HomeTeamMoneyLine
0,202111304,13,NE,1,BUF,1,-0.8,11.2,36,-42
1,202111304,13,NE,1,BUF,0,-0.8,11.2,36,-42
2,202111304,13,NE,0,BUF,0,-0.8,11.2,36,-42
3,202111304,13,NE,1,BUF,1,-0.8,11.2,36,-42
4,202111304,13,NE,1,BUF,1,-0.8,11.2,36,-42
5,202111304,13,NE,1,BUF,1,-0.8,11.2,36,-42
6,202111304,13,NE,1,BUF,1,-0.8,11.2,36,-42
7,202111304,13,NE,0,BUF,0,-0.8,11.2,36,-42
8,202111304,13,NE,0,BUF,0,-0.8,11.2,36,-42
9,202111304,13,NE,1,BUF,1,-0.8,11.2,36,-42


##### Business Question: Is the current score covering the point spread?

In [14]:
sep = pd_games_stream['AwayScore'].max() - pd_games_stream['HomeScore'].max()
answer = sep > pd_games_stream['PointSpread'].max()
answer

True

##### Business Question: With the current score, have I hit the over?

In [15]:
total_score = pd_games_stream['AwayScore'].max() + pd_games_stream['HomeScore'].max()
answer2 = total_score > pd_games_stream['OverUnder'].max()
answer2

False

# Pull in historical batch data

In [9]:
# Read data from parquet
batch_df = spark.read.parquet('/tmp/season')

# Create table for queries
batch_df.registerTempTable('season')

# Pull necessary data and turn into table - saving back into parquet
batch_query = """
create external table my_season
  stored as parquet
  location '/tmp/season_out'
  as
  select * from season
"""
spark.sql(batch_query)

DataFrame[]

In [10]:
# Read table and create dataframe for analysis
season = spark.read.parquet('/tmp/season_out')
season_df = season.toPandas()

# Drop NAs that represent future weeks of the season
season_df = season_df.dropna()
season_df

,GameKey,Week,AwayTeam,AwayScore,HomeTeam,HomeScore,PointSpread,OverUnder,AwayTeamMoneyLine,HomeTeamMoneyLine
0,202110133,1,DAL,37.0,TB,39.0,-11.400000,66.699997,433.0,-558.0
1,202110102,1,PHI,41.0,ATL,10.0,-4.400000,61.599998,188.0,-226.0
2,202110104,1,PIT,29.0,BUF,20.0,-8.300000,61.000000,294.0,-358.0
3,202110105,1,NYJ,18.0,CAR,24.0,-5.100000,57.200001,198.0,-239.0
4,202110107,1,MIN,31.0,CIN,34.0,4.800000,60.400002,-194.0,163.0
5,202110111,1,SF,52.0,DET,42.0,12.100000,58.500000,-544.0,421.0
6,202110113,1,JAX,27.0,HOU,47.0,5.700000,57.799999,-216.0,180.0
7,202110114,1,SEA,36.0,IND,20.0,4.800000,61.599998,-197.0,164.0
8,202110134,1,ARI,48.0,TEN,17.0,-3.200000,68.599998,156.0,-186.0
9,202110135,1,LAC,25.0,WAS,20.0,-2.500000,57.799999,139.0,-164.0


In [134]:
# Add additonal columns to spur betting analysis
season_df["TotalScore"] = season_df["HomeScore"] + season_df["AwayScore"]
season_df["ScoreDiff"] = season_df["TotalScore"] - season_df["OverUnder"]
season_df["FinalSpread"] = (season_df["HomeScore"] - season_df["AwayScore"])*(-1)
season_df["SpreadDiff"] = season_df["FinalSpread"] - season_df["PointSpread"]
season_df["SpreadDiff"] = season_df["SpreadDiff"].abs()

In [135]:
# Functions to effectively query data

# Query by week
def week(number):
    week_seas = season_df[season_df["Week"] == number]
    return week_seas

# Query by team
def season(team):
    team_seas = season_df[(season_df["AwayTeam"] == team) | (season_df["HomeTeam"] == team)]
    return team_seas

## Let's look at how the 49ers have performed this season against the spread, over/under, and ML?

In [136]:
Niners = season("SF")
Niners

,GameKey,Week,AwayTeam,AwayScore,HomeTeam,HomeScore,PointSpread,OverUnder,AwayTeamMoneyLine,HomeTeamMoneyLine,TotalScore,ScoreDiff,FinalSpread,SpreadDiff
5,202110111,1,SF,52.0,DET,42.0,12.1,58.500000,-544.0,421.0,94.0,35.500000,10.0,2.1
24,202110226,2,SF,22.0,PHI,14.0,4.8,62.299999,-189.0,159.0,36.0,-26.299999,8.0,3.2
46,202110331,3,GB,38.0,SF,36.0,-4.4,64.199997,188.0,-224.0,74.0,9.800003,2.0,6.4
59,202110431,4,SEA,36.0,SF,27.0,-3.2,66.099998,146.0,-174.0,63.0,-3.099998,9.0,12.2
76,202110501,5,SF,13.0,ARI,22.0,-7.0,61.599998,253.0,-309.0,35.0,-26.599998,-9.0,2.0
105,202110731,7,IND,38.0,SF,23.0,-4.4,52.700001,184.0,-224.0,61.0,8.299999,15.0,19.4
110,202110806,8,SF,42.0,CHI,28.0,6.5,50.799999,-262.0,219.0,70.0,19.200001,14.0,7.5
133,202110931,9,ARI,39.0,SF,22.0,-7.0,56.599998,257.0,-313.0,61.0,4.400002,17.0,24.0
149,202111031,10,LAR,13.0,SF,39.0,5.7,64.199997,-233.0,194.0,52.0,-12.199997,-26.0,31.7
155,202111115,11,SF,38.0,JAX,13.0,10.5,57.799999,-365.0,296.0,51.0,-6.799999,25.0,14.5


In [141]:
# How many games hit the OVER
OVER = Niners[Niners["ScoreDiff"] > 0].count()
OVER = OVER["ScoreDiff"]
print(OVER, "total 49ers games hit the over the predicted point total!")

# How many games hit the UNDER
UNDER = Niners[Niners["ScoreDiff"] < 0].count()
UNDER = UNDER["ScoreDiff"]
print(UNDER, "total 49ers games went under the predicted point total!")

7 total 49ers games hit the over the predicted point total!
5 total 49ers games went under the predicted point total!


In [142]:
# Create money line column for just SF
conditions = [
    (Niners["AwayTeam"] == "SF"),
    (Niners["HomeTeam"] == "SF")]

values = [Niners["AwayTeamMoneyLine"], Niners["HomeTeamMoneyLine"]]

Niners["MoneyLine"] = np.select(conditions, values)

# Create spread column for just SF
conditions = [
    (Niners["AwayTeam"] == "SF"),
    (Niners["HomeTeam"] == "SF")]

values1 = [Niners["PointSpread"]*(-1), Niners["PointSpread"]]

Niners["Spread"] = np.select(conditions, values1)

# Create point column for just SF
conditions = [
    (Niners["AwayTeam"] == "SF"),
    (Niners["HomeTeam"] == "SF")]

values2 = [Niners["AwayScore"], Niners["HomeScore"]]

Niners["NinersScore"] = np.select(conditions, values2)

# Create point column for opponent
conditions = [
    (Niners["AwayTeam"] != "SF"),
    (Niners["HomeTeam"] != "SF")]

values3 = [Niners["AwayScore"], Niners["HomeScore"]]

Niners["OppScore"] = np.select(conditions, values3)


/opt/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/opt/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/opt/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#inde

In [144]:
# How many games did the 49ers cover the spread
Niners["SpreadFinal"] = Niners["NinersScore"] - Niners["OppScore"]
covered = Niners[Niners["SpreadFinal"] > 0].count()
covered = covered["SpreadFinal"]
print("The 49ers covered in", covered, "total games!")

The 49ers covered in 6 total games!


/opt/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [145]:
# Moneyline
ML_DF1 = Niners[(Niners["MoneyLine"] < 0) & (Niners["SpreadFinal"] >0)]
hit_ML = ML_DF1["MoneyLine"].count()
print("The 49ers were favored to win and actually won in", hit_ML, "total games!")

# Moneyline
ML_DF2 = Niners[(Niners["MoneyLine"] > 0) & (Niners["SpreadFinal"] >0)]
hit_ML_un = ML_DF2["MoneyLine"].count()
print("The 49ers were favored to lose and actually won in", hit_ML_un, "total games!")

# Moneyline
ML_DF3 = Niners[(Niners["MoneyLine"] < 0) & (Niners["SpreadFinal"] <0)]
no_ML = ML_DF3["MoneyLine"].count()
print("The 49ers were favored to win and actually lost in", no_ML, "total games!")

# Moneyline
ML_DF4 = Niners[(Niners["MoneyLine"] > 0) & (Niners["SpreadFinal"] <0)]
noun_ML = ML_DF4["MoneyLine"].count()
print("The 49ers were favored to lose and actually lost in", noun_ML, "total games!")

The 49ers were favored to win and actually won in 5 total games!
The 49ers were favored to lose and actually won in 1 total games!
The 49ers were favored to win and actually lost in 5 total games!
The 49ers were favored to lose and actually lost in 1 total games!


### At the conclusion of Week 13, the 49ers having the following betting stats:
- 6-6 against the spread
- 7-5 hit for over for total point prediction
- Have only won 5 out of 10 games in which they were favored

## Comparing results to predictions

In [127]:
# Top 10 biggest misses
bet_misses = season_df.sort_values(by = "SpreadDiff", ascending = False)
bet_misses = bet_misses[["AwayTeam", "AwayScore", "HomeTeam", "HomeScore", "PointSpread", "SpreadDiff"]]

### Wow - did Vegas get it wrong! Here are the top 10 games with largest different in spread and outcome:

In [126]:
bet_misses.head(10)

,AwayTeam,AwayScore,HomeTeam,HomeScore,PointSpread,SpreadDiff
12,GB,5.0,NO,48.0,5.7,48.7
98,NYJ,17.0,NE,69.0,-8.9,43.1
139,CLE,11.0,NE,57.0,-3.2,42.8
151,IND,52.0,BUF,19.0,-8.9,41.9
112,PHI,56.0,DET,10.0,5.7,40.3
137,ATL,5.0,DAL,55.0,-10.2,39.8
144,CAR,43.0,ARI,13.0,-8.9,38.9
22,BUF,44.0,MIA,0.0,5.7,38.3
95,CIN,52.0,BAL,22.0,-8.3,38.3
100,KC,5.0,TEN,34.0,6.5,35.5


In [128]:
# Biggest home upsets based on ML
ML_Away = season_df[(season_df["AwayTeamMoneyLine"] > 0) & (season_df["FinalSpread"] >0)]
ML_Away = ML_Away.sort_values(by='AwayTeamMoneyLine', ascending=False)

### Cowboys and Bills are inconsistent! Here are the biggest upsets for home teams:

In [129]:
ML_Away.head(10)

,GameKey,Week,AwayTeam,AwayScore,HomeTeam,HomeScore,PointSpread,OverUnder,AwayTeamMoneyLine,HomeTeamMoneyLine,TotalScore,ScoreDiff,FinalSpread,SpreadDiff
126,202110909,9,DEN,38.0,DAL,20.0,-12.7,63.599998,464.0,-608.0,58.0,-5.599998,18.0,30.7
159,202111134,11,HOU,28.0,TEN,17.0,-12.7,57.200001,460.0,-597.0,45.0,-12.200001,11.0,23.7
134,202110932,9,TEN,36.0,LAR,20.0,-8.9,67.400002,350.0,-437.0,56.0,-11.400002,16.0,24.9
55,202110422,4,NYG,34.0,NO,27.0,-8.9,54.000000,343.0,-430.0,61.0,7.000000,7.0,15.9
166,202111209,12,LV,46.0,DAL,42.0,-8.9,64.199997,334.0,-419.0,88.0,23.800003,4.0,12.9
37,202110316,3,LAC,38.0,KC,31.0,-8.9,68.599998,315.0,-388.0,69.0,0.400002,7.0,15.9
151,202111104,11,IND,52.0,BUF,19.0,-8.9,62.900002,314.0,-393.0,71.0,8.099998,33.0,41.9
144,202111001,10,CAR,43.0,ARI,13.0,-8.9,52.700001,313.0,-388.0,56.0,3.299999,30.0,38.9
2,202110104,1,PIT,29.0,BUF,20.0,-8.3,61.000000,294.0,-358.0,49.0,-12.000000,9.0,17.3
29,202110230,2,TEN,42.0,SEA,38.0,-8.3,68.599998,286.0,-357.0,80.0,11.400002,4.0,12.3


In [125]:
# Biggest away upsets based on ML
ML_home = season_df[(season_df["HomeTeamMoneyLine"] > 0) & (season_df["FinalSpread"] <0)]
ML_home = ML_home.sort_values(by='HomeTeamMoneyLine', ascending=False)

,GameKey,Week,AwayTeam,AwayScore,HomeTeam,HomeScore,PointSpread,OverUnder,AwayTeamMoneyLine,HomeTeamMoneyLine,TotalScore,ScoreDiff,FinalSpread,SpreadDiff
127,202110915,9,BUF,10.0,JAX,11.0,20.299999,61.599998,-1526.0,928.0,21.0,-40.599998,-1.0,21.299999
115,202110824,8,CIN,39.0,NYJ,43.0,14.600000,54.700001,-722.0,534.0,82.0,27.299999,-4.0,18.600000
143,202111035,10,TB,24.0,WAS,37.0,12.100000,64.199997,-564.0,437.0,61.0,-3.199997,-13.0,25.100000
136,202111019,10,BAL,13.0,MIA,28.0,10.800000,58.500000,-525.0,414.0,41.0,-17.500000,-15.0,25.800000
184,202111311,13,MIN,34.0,DET,37.0,11.300000,59.700001,-395.0,319.0,71.0,11.299999,-3.0,14.300000
93,202110634,6,BUF,39.0,TEN,43.0,9.700000,68.000000,-342.0,280.0,82.0,14.000000,-4.0,13.700000
56,202110424,4,TEN,31.0,NYJ,34.0,8.900000,56.599998,-339.0,273.0,65.0,8.400002,-3.0,11.900000
190,202111328,13,BAL,24.0,PIT,25.0,6.500000,55.900002,-254.0,214.0,49.0,-6.900002,-1.0,7.500000
100,202110734,7,KC,5.0,TEN,34.0,6.500000,74.400002,-254.0,212.0,39.0,-35.400002,-29.0,35.500000
119,202110822,8,TB,34.0,NO,46.0,6.500000,61.599998,-253.0,212.0,80.0,18.400002,-12.0,18.500000


### The Bills keep getting upset! Bet against them! Here are the top 10 biggest away upsets:

In [130]:
ML_home.head(10)

,GameKey,Week,AwayTeam,AwayScore,HomeTeam,HomeScore,PointSpread,OverUnder,AwayTeamMoneyLine,HomeTeamMoneyLine,TotalScore,ScoreDiff,FinalSpread,SpreadDiff
127,202110915,9,BUF,10.0,JAX,11.0,20.299999,61.599998,-1526.0,928.0,21.0,-40.599998,-1.0,21.299999
115,202110824,8,CIN,39.0,NYJ,43.0,14.600000,54.700001,-722.0,534.0,82.0,27.299999,-4.0,18.600000
143,202111035,10,TB,24.0,WAS,37.0,12.100000,64.199997,-564.0,437.0,61.0,-3.199997,-13.0,25.100000
136,202111019,10,BAL,13.0,MIA,28.0,10.800000,58.500000,-525.0,414.0,41.0,-17.500000,-15.0,25.800000
184,202111311,13,MIN,34.0,DET,37.0,11.300000,59.700001,-395.0,319.0,71.0,11.299999,-3.0,14.300000
93,202110634,6,BUF,39.0,TEN,43.0,9.700000,68.000000,-342.0,280.0,82.0,14.000000,-4.0,13.700000
56,202110424,4,TEN,31.0,NYJ,34.0,8.900000,56.599998,-339.0,273.0,65.0,8.400002,-3.0,11.900000
190,202111328,13,BAL,24.0,PIT,25.0,6.500000,55.900002,-254.0,214.0,49.0,-6.900002,-1.0,7.500000
100,202110734,7,KC,5.0,TEN,34.0,6.500000,74.400002,-254.0,212.0,39.0,-35.400002,-29.0,35.500000
119,202110822,8,TB,34.0,NO,46.0,6.500000,61.599998,-253.0,212.0,80.0,18.400002,-12.0,18.500000
